# HandTrack: Real-Time Finger Counting with OpenCV

This project implements a real-time finger counter using **OpenCV**. It detects and counts the number of visible fingers in a specified region of interest (ROI) from a webcam feed. The program uses image processing techniques such as:

- **Background subtraction** for isolating movements.
- **Thresholding** and **contour detection** for hand segmentation.
- **Convex hull analysis** to count visible fingers.

### Features:
- Real-time finger counting and hand contour visualization.
- Adaptive to different hand positions using dynamic ROI.
- Simple and effective for gesture-based applications.

### Tech Stack:
- Python, OpenCV, NumPy, scikit-learn

This project is a great introduction to computer vision concepts and their practical applications.


In [2]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

In [3]:
# Global Variables
background = None
accumulated_weight = 0.5

# Corners of the rectangle
roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [9]:
# Function that finds the average backgroung value
def calc_accum_avg(frame, accumulated_weight):
    global background
    if background is None:
        # Initialize the background
        background = frame.copy().astype('float')
        return None

    # Update the background with a weighted average
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [10]:
# Function for the thresholding and extrernal contours
def segment(frame, threshold=25):
    # Compute the absolute difference between the background and the current frame
    diff = cv2.absdiff(background.astype('uint8'), frame)

    # Apply thresholding to the difference image
    _, thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

    # Handle cv2.findContours for different OpenCV versions
    contours_result = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Unpack based on the number of return values
    if len(contours_result) == 3:
        _, contours, hierarchy = contours_result  # OpenCV 3.x
    else:
        contours, hierarchy = contours_result  # OpenCV 4.x

    if len(contours) == 0:
        return None
    else:
        # Assuming the largest external contour in the ROI is the hand
        hand_segment = max(contours, key=cv2.contourArea)

        return (thresholded, hand_segment)


In [16]:
# Counting fingers with a Convex Hull - calculating the most external points and drawing a polygon around  it
import numpy as np
from sklearn.metrics import pairwise

def count_fingers(thresholded, hand_segment):
    # Compute the convex hull
    conv_hull = cv2.convexHull(hand_segment)

    # Identify extreme points
    top = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])    # Top point
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])  # Bottom point
    left = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])    # Leftmost point
    right = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])   # Rightmost point

    # Center of the hand
    cX = (left[0] + right[0]) // 2  # Center X
    cY = (top[1] + bottom[1]) // 2  # Center Y

    # Compute distances from the center to the extreme points
    distances = pairwise.euclidean_distances([(cX, cY)], Y=[left, right, top, bottom])[0]
    max_distance = distances.max()

    # Define a circular ROI
    radius = int(0.6 * max_distance)  # Radius of the circular ROI
    circumference = (2 * np.pi * radius)

    # Create a mask for the circular ROI
    circular_roi = np.zeros(thresholded.shape, dtype="uint8")
    cv2.circle(circular_roi, (cX, cY), radius, 255, 10)

    # Apply the circular ROI as a mask on the thresholded image
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

    # Handle different OpenCV versions for findContours
    contours_result = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours_result) == 3:
        _, contours, hierarchy = contours_result  # OpenCV 3.x
    else:
        contours, hierarchy = contours_result  # OpenCV 4.x

    count = 0  # Initialize finger count

    for cnt in contours:
        (x, y, w, h) = cv2.boundingRect(cnt)

        # Conditions for counting fingers
        out_of_wrist = (cY + (cY * 0.25)) > (y + h)  # Contour should not include wrist
        limit_points = (circumference * 0.25) > cnt.shape[0]  # Limit small contours (noise)

        if out_of_wrist and limit_points:  # If both conditions are met, it's a finger
            count += 1

    return count

In [17]:
num_frames = 0
cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    frame_copy = frame.copy()
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(7,7),0)
    
    if num_frames < 60:
        calc_accum_avg(gray, accumulated_weight)
        if num_frames <=59:
            cv2.putText(frame_copy, 'Wait. Getting Background.', (200,300), cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow('Finger Count', frame_copy)
            
    else: 
        hand = segment(gray)
        
        if hand is not None:
            thresholded, hand_segment = hand 
            
            cv2.drawContours(frame_copy, [hand_segment+(roi_right, roi_top)],-1,(255,0,0),5) # draws contours around real hand in live stream
            
            fingers = count_fingers(thresholded, hand_segment)
            
            cv2.putText(frame_copy, str(fingers), (70,50), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,1,(0,0,255),2)
            
            cv2.imshow('Thresholded', thresholded)
            
    cv2.rectangle(frame_copy,(roi_left, roi_top),(roi_right, roi_bottom), (0,0,255), 5)
    
    num_frames +=1 
    
    cv2.imshow('Finger Count', frame_copy)
    
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break

cam.release()
cv2.destroyAllWindows() 